# Mutual Fund Backtest: Top 5 by 5-Year Rolling Return

This notebook performs a backtest on mutual fund data with the following rules:

1. Start investment on 01-01-2018 with 500,000 INR.
2. At each rebalance (every year), select the top 5 funds with the highest 5-year rolling return, **only considering funds with at least 5 years of NAV history at that point**.
3. Divide the investment equally among the selected funds.
4. If a fund drops out of the top 5, exit and reinvest in the new top fund.
5. Rebalance annually and calculate the final return as of the latest available date.
6. Save a readable log of all rebalances as both CSV and JSON.

In [ ]:
import pandas as pd
import json

# Load data
df = pd.read_csv('mf_data.csv', parse_dates=['date'])

# Backtest parameters
start_date = pd.Timestamp('2019-01-01')
end_date = df['date'].max()
initial_investment = 1000000
rebalance_dates = pd.date_range(start=start_date, end=end_date, freq='12MS')

# portfolio: {fund: {'units': float, 'buy_nav': float, 'buy_date': Timestamp, 'last_nav': float}}
portfolio = {}
portfolio_value = initial_investment
rebalance_log = []

for i, rebalance_date in enumerate(rebalance_dates):
    five_years_ago = rebalance_date - pd.DateOffset(years=5)
    rolling_returns = {}
    # Only consider funds with NAV on or before both rebalance_date and five_years_ago
    for fund in df['Scheme Name'].unique():
        fund_df = df[df['Scheme Name'] == fund]
        nav_now_row = fund_df[fund_df['date'] <= rebalance_date].tail(1)
        nav_past_row = fund_df[fund_df['date'] <= five_years_ago].tail(1)
        if not nav_now_row.empty and not nav_past_row.empty:
            nav_now = nav_now_row['nav'].values[0]
            nav_past = nav_past_row['nav'].values[0]
            rolling_return = (nav_now / nav_past) - 1
            rolling_returns[fund] = rolling_return

    # Select top 3 funds by rolling return
    top3 = sorted(rolling_returns, key=rolling_returns.get, reverse=True)[:3]
    top3_set = set(top3)

    # Get NAVs for rebalance date
    navs = {}
    for fund in top3:
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        if not nav_row.empty:
            navs[fund] = nav_row['nav'].values[0]

    # Determine funds to sell and buy
    sold_funds = list(set(portfolio.keys()) - top3_set)
    bought_funds = list(top3_set - set(portfolio.keys()))
    held_funds = list(top3_set & set(portfolio.keys()))

    # Calculate profit/loss for sold funds using correct buy_nav and buy_date
    pl_report = []
    for fund in sold_funds:
        units = portfolio[fund]['units']
        buy_nav = portfolio[fund]['buy_nav']
        buy_date = portfolio[fund]['buy_date']
        # Sell NAV is NAV at rebalance_date
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        sell_nav = nav_row['nav'].values[0] if not nav_row.empty else None
        pl = (sell_nav - buy_nav) * units if sell_nav is not None else None
        pl_report.append({'fund': fund, 'units': units, 'buy_nav': buy_nav, 'buy_date': str(buy_date), 'sell_nav': sell_nav, 'sell_date': str(rebalance_date), 'profit_loss': pl})

    # Calculate value of held funds (not sold)
    held_value = 0
    for fund in held_funds:
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        if not nav_row.empty:
            nav = nav_row['nav'].values[0]
            held_value += portfolio[fund]['units'] * nav

    # Add value from sold funds (i.e., proceeds from sale)
    sold_value = 0
    for fund in sold_funds:
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        if not nav_row.empty:
            nav = nav_row['nav'].values[0]
            sold_value += portfolio[fund]['units'] * nav

    # The new portfolio value is the sum of held and sold values (i.e., after selling exited funds, before buying new funds)
    portfolio_value = held_value + sold_value

    # Allocate equally to top3
    new_portfolio = {}
    if navs:
        amount_per_fund = portfolio_value / len(navs)
        for fund, nav in navs.items():
            if fund in portfolio:
                # If already in portfolio, keep original buy_nav and buy_date, update units to new allocation
                prev = portfolio[fund]
                units = amount_per_fund / nav
                new_portfolio[fund] = {'units': units, 'buy_nav': prev['buy_nav'], 'buy_date': prev['buy_date'], 'last_nav': nav}
            else:
                # New fund, set buy_nav and buy_date
                units = amount_per_fund / nav
                new_portfolio[fund] = {'units': units, 'buy_nav': nav, 'buy_date': rebalance_date, 'last_nav': nav}

    # Log this rebalance
    rebalance_log.append({
        'rebalance_date': str(rebalance_date.date()),
        'top3': [(fund, round(rolling_returns[fund]*100,2)) for fund in top3],
        'bought_funds': bought_funds,
        'sold_funds': sold_funds,
        'pl_report': pl_report,
        'portfolio_value': round(portfolio_value,2)
    })

    portfolio = new_portfolio

# Final update for last period
final_value = sum([v['units'] * v['last_nav'] for v in portfolio.values()])
today = df['date'].max()
print(f"\nFinal portfolio value as of {today.date()}: {final_value:.2f} INR")
print(f"Total return: {((final_value/initial_investment - 1)*100):.2f}%")


Final portfolio value as of 2025-06-13: 0.00 INR
Total return: -100.00%


In [ ]:
# Save readable rebalance log
readable_rows = []
for log in rebalance_log:
    # Expand pl_report for each sold fund, or add a row with empty P/L if none sold
    if log['pl_report']:
        for pl in log['pl_report']:
            readable_rows.append({
                'rebalance_date': log['rebalance_date'],
                'top3': ', '.join([f'{f} ({ret}%)' for f, ret in log['top3']]),
                'bought_funds': ', '.join(log['bought_funds']),
                'sold_funds': ', '.join(log['sold_funds']),
                'sold_fund': pl['fund'],
                'units': round(pl['units'], 2),
                'buy_nav': round(pl['buy_nav'], 2),
                'buy_date': pl['buy_date'],
                'sell_nav': round(pl['sell_nav'], 2) if pl['sell_nav'] is not None else '',
                'sell_date': pl['sell_date'],
                'profit_loss': round(pl['profit_loss'], 2) if pl['profit_loss'] is not None else '',
                'portfolio_value': log['portfolio_value']
            })
    else:
        readable_rows.append({
            'rebalance_date': log['rebalance_date'],
            'top3': ', '.join([f'{f} ({ret}%)' for f, ret in log['top3']]),
            'bought_funds': ', '.join(log['bought_funds']),
            'sold_funds': ', '.join(log['sold_funds']),
            'sold_fund': '',
            'units': '',
            'buy_nav': '',
            'buy_date': '',
            'sell_nav': '',
            'sell_date': '',
            'profit_loss': '',
            'portfolio_value': log['portfolio_value']
        })

pd.DataFrame(readable_rows).to_csv('rebalance_log_readable.csv', index=False)
# Convert all date objects in rebalance_log to string for JSON serialization
def convert_dates(obj):
    if isinstance(obj, dict):
        return {k: convert_dates(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_dates(i) for i in obj]
    elif hasattr(obj, 'isoformat'):
        return obj.isoformat()
    else:
        return obj

rebalance_log_serializable = convert_dates(rebalance_log)
with open('rebalance_log_readable.json', 'w', encoding='utf-8') as f:
    json.dump(rebalance_log_serializable, f, indent=2, ensure_ascii=False)
print('Readable rebalance log saved to rebalance_log_readable.csv and rebalance_log_readable.json')

Readable rebalance log saved to rebalance_log_readable.csv and rebalance_log_readable.json


In [ ]:
import pandas as pd
import json

df = pd.read_csv('mf_data.csv', parse_dates=['date'])

start_date = pd.Timestamp('2019-01-01')
end_date = df['date'].max()
initial_investment = 1000000
rebalance_dates = pd.date_range(start=start_date, end=end_date, freq='12MS')

portfolio = {}
rebalance_log = []
cash = initial_investment

for i, rebalance_date in enumerate(rebalance_dates):
    five_years_ago = rebalance_date - pd.DateOffset(years=1)
    rolling_returns = {}
    for fund in df['Scheme Name'].unique():
        fund_df = df[df['Scheme Name'] == fund]
        nav_now_row = fund_df[fund_df['date'] <= rebalance_date].tail(1)
        nav_past_row = fund_df[fund_df['date'] <= five_years_ago].tail(1)
        if not nav_now_row.empty and not nav_past_row.empty:
            nav_now = nav_now_row['nav'].values[0]
            nav_past = nav_past_row['nav'].values[0]
            rolling_return = (nav_now / nav_past) - 1
            rolling_returns[fund] = rolling_return

    top3 = sorted(rolling_returns, key=rolling_returns.get, reverse=True)[:3]
    top3_set = set(top3)

    navs = {}
    for fund in top3:
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        if not nav_row.empty:
            navs[fund] = nav_row['nav'].values[0]

    portfolio_value = cash
    portfolio_value_breakdown = []
    for fund, pos in portfolio.items():
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        if not nav_row.empty:
            nav = nav_row['nav'].values[0]
            portfolio[fund]['last_nav'] = nav
            value = pos['units'] * nav
            portfolio_value += value
            portfolio_value_breakdown.append({
                'fund': fund,
                'units': pos['units'],
                'nav': nav,
                'value': value
            })
        else:
            portfolio[fund]['last_nav'] = None
            portfolio_value_breakdown.append({
                'fund': fund,
                'units': pos['units'],
                'nav': None,
                'value': 0
            })
    if cash > 0:
        portfolio_value_breakdown.append({'fund': 'CASH', 'units': '', 'nav': '', 'value': cash})

    sold_funds = list(set(portfolio.keys()) - top3_set)
    pl_report = []
    for fund in sold_funds:
        pos = portfolio[fund]
        sell_nav = pos['last_nav']
        if sell_nav is not None:
            proceeds = pos['units'] * sell_nav
            cash += proceeds
            pl = (sell_nav - pos['buy_nav']) * pos['units']
        else:
            proceeds = 0
            pl = None
        pl_report.append({
            'fund': fund,
            'units': pos['units'],
            'buy_nav': pos['buy_nav'],
            'buy_date': str(pos['buy_date']),
            'sell_nav': sell_nav,
            'sell_date': str(rebalance_date),
            'profit_loss': pl
        })
        del portfolio[fund]

    held_funds = list(top3_set & set(portfolio.keys()))
    held_value = 0
    for fund in held_funds:
        nav = portfolio[fund]['last_nav']
        held_value += portfolio[fund]['units'] * nav

    total_to_invest = cash + held_value
    amount_per_fund = total_to_invest / 3 if len(navs) == 3 else 0

    new_portfolio = {}
    bought_funds = []
    for fund in top3:
        nav = navs[fund]
        if fund in portfolio:
            units = amount_per_fund / nav
            new_portfolio[fund] = {
                'units': units,
                'buy_nav': portfolio[fund]['buy_nav'],
                'buy_date': portfolio[fund]['buy_date'],
                'last_nav': nav
            }
        else:
            units = amount_per_fund / nav
            new_portfolio[fund] = {
                'units': units,
                'buy_nav': nav,
                'buy_date': rebalance_date,
                'last_nav': nav
            }
            bought_funds.append(fund)
    portfolio = new_portfolio
    cash = 0

    rebalance_log.append({
        'rebalance_date': str(rebalance_date.date()),
        'top3': [(fund, round(rolling_returns[fund]*100,2)) for fund in top3],
        'bought_funds': bought_funds,
        'sold_funds': sold_funds,
        'pl_report': pl_report,
        'portfolio_value': round(sum([v['units'] * v['last_nav'] for v in portfolio.values()]),2),
        'portfolio_value_breakdown': portfolio_value_breakdown
    })

final_value = sum([v['units'] * v['last_nav'] for v in portfolio.values()])
today = df['date'].max()
print(f"\nFinal portfolio value as of {today.date()}: {final_value:.2f} INR")
print(f"Total return: {((final_value/initial_investment - 1)*100):.2f}%")

# Save readable rebalance log
readable_rows = []
for log in rebalance_log:
    breakdown_str = json.dumps(log['portfolio_value_breakdown'])
    if log['pl_report']:
        for pl in log['pl_report']:
            readable_rows.append({
                'rebalance_date': log['rebalance_date'],
                'top3': ', '.join([f'{f} ({ret}%)' for f, ret in log['top3']]),
                'bought_funds': ', '.join(log['bought_funds']),
                'sold_funds': ', '.join(log['sold_funds']),
                'sold_fund': pl['fund'],
                'units': round(pl['units'], 2),
                'buy_nav': round(pl['buy_nav'], 2),
                'buy_date': pl['buy_date'],
                'sell_nav': round(pl['sell_nav'], 2) if pl['sell_nav'] is not None else '',
                'sell_date': pl['sell_date'],
                'profit_loss': round(pl['profit_loss'], 2) if pl['profit_loss'] is not None else '',
                'portfolio_value': log['portfolio_value'],
                'portfolio_value_breakdown': breakdown_str
            })
    else:
        readable_rows.append({
            'rebalance_date': log['rebalance_date'],
            'top3': ', '.join([f'{f} ({ret}%)' for f, ret in log['top3']]),
            'bought_funds': ', '.join(log['bought_funds']),
            'sold_funds': ', '.join(log['sold_funds']),
            'sold_fund': '',
            'units': '',
            'buy_nav': '',
            'buy_date': '',
            'sell_nav': '',
            'sell_date': '',
            'profit_loss': '',
            'portfolio_value': log['portfolio_value'],
            'portfolio_value_breakdown': breakdown_str
        })

pd.DataFrame(readable_rows).to_csv('rebalance_log_readable.csv', index=False)
def convert_dates(obj):
    if isinstance(obj, dict):
        return {k: convert_dates(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_dates(i) for i in obj]
    elif hasattr(obj, 'isoformat'):
        return obj.isoformat()
    else:
        return obj

rebalance_log_serializable = convert_dates(rebalance_log)
with open('rebalance_log_readable.json', 'w', encoding='utf-8') as f:
    json.dump(rebalance_log_serializable, f, indent=2, ensure_ascii=False)
print('Readable rebalance log saved to rebalance_log_readable.csv and rebalance_log_readable.json')



Final portfolio value as of 2025-06-13: 3458864.93 INR
Total return: 245.89%
Readable rebalance log saved to rebalance_log_readable.csv and rebalance_log_readable.json


In [ ]:
import pandas as pd
import json

df = pd.read_csv('mf_data.csv', parse_dates=['date'])

start_date = pd.Timestamp('2019-01-01')
end_date = df['date'].max()
initial_investment = 1000000
# Rebalance every 6 months
rebalance_dates = pd.date_range(start=start_date, end=end_date, freq='6MS')

portfolio = {}
rebalance_log = []
cash = initial_investment

for i, rebalance_date in enumerate(rebalance_dates):
    five_years_ago = rebalance_date - pd.DateOffset(years=1)
    rolling_returns = {}
    for fund in df['Scheme Name'].unique():
        fund_df = df[df['Scheme Name'] == fund]
        nav_now_row = fund_df[fund_df['date'] <= rebalance_date].tail(1)
        nav_past_row = fund_df[fund_df['date'] <= five_years_ago].tail(1)
        if not nav_now_row.empty and not nav_past_row.empty:
            nav_now = nav_now_row['nav'].values[0]
            nav_past = nav_past_row['nav'].values[0]
            rolling_return = (nav_now / nav_past) - 1
            rolling_returns[fund] = rolling_return

    top3 = sorted(rolling_returns, key=rolling_returns.get, reverse=True)[:3]
    top3_set = set(top3)

    navs = {}
    for fund in top3:
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        if not nav_row.empty:
            navs[fund] = nav_row['nav'].values[0]

    portfolio_value = cash
    portfolio_value_breakdown = []
    for fund, pos in portfolio.items():
        nav_row = df[(df['Scheme Name'] == fund) & (df['date'] <= rebalance_date)].tail(1)
        if not nav_row.empty:
            nav = nav_row['nav'].values[0]
            portfolio[fund]['last_nav'] = nav
            value = pos['units'] * nav
            portfolio_value += value
            portfolio_value_breakdown.append({
                'fund': fund,
                'units': pos['units'],
                'nav': nav,
                'value': value
            })
        else:
            portfolio[fund]['last_nav'] = None
            portfolio_value_breakdown.append({
                'fund': fund,
                'units': pos['units'],
                'nav': None,
                'value': 0
            })
    if cash > 0:
        portfolio_value_breakdown.append({'fund': 'CASH', 'units': '', 'nav': '', 'value': cash})

    sold_funds = list(set(portfolio.keys()) - top3_set)
    pl_report = []
    for fund in sold_funds:
        pos = portfolio[fund]
        sell_nav = pos['last_nav']
        if sell_nav is not None:
            proceeds = pos['units'] * sell_nav
            cash += proceeds
            pl = (sell_nav - pos['buy_nav']) * pos['units']
        else:
            proceeds = 0
            pl = None
        pl_report.append({
            'fund': fund,
            'units': pos['units'],
            'buy_nav': pos['buy_nav'],
            'buy_date': str(pos['buy_date']),
            'sell_nav': sell_nav,
            'sell_date': str(rebalance_date),
            'profit_loss': pl
        })
        del portfolio[fund]

    held_funds = list(top3_set & set(portfolio.keys()))
    held_value = 0
    for fund in held_funds:
        nav = portfolio[fund]['last_nav']
        held_value += portfolio[fund]['units'] * nav

    total_to_invest = cash + held_value
    amount_per_fund = total_to_invest / 3 if len(navs) == 3 else 0

    new_portfolio = {}
    bought_funds = []
    for fund in top3:
        nav = navs[fund]
        if fund in portfolio:
            units = amount_per_fund / nav
            new_portfolio[fund] = {
                'units': units,
                'buy_nav': portfolio[fund]['buy_nav'],
                'buy_date': portfolio[fund]['buy_date'],
                'last_nav': nav
            }
        else:
            units = amount_per_fund / nav
            new_portfolio[fund] = {
                'units': units,
                'buy_nav': nav,
                'buy_date': rebalance_date,
                'last_nav': nav
            }
            bought_funds.append(fund)
    portfolio = new_portfolio
    cash = 0

    rebalance_log.append({
        'rebalance_date': str(rebalance_date.date()),
        'top3': [(fund, round(rolling_returns[fund]*100,2)) for fund in top3],
        'bought_funds': bought_funds,
        'sold_funds': sold_funds,
        'pl_report': pl_report,
        'portfolio_value': round(sum([v['units'] * v['last_nav'] for v in portfolio.values()]),2),
        'portfolio_value_breakdown': portfolio_value_breakdown
    })

final_value = sum([v['units'] * v['last_nav'] for v in portfolio.values()])
today = df['date'].max()
print(f"\nFinal portfolio value as of {today.date()}: {final_value:.2f} INR")
print(f"Total return: {((final_value/initial_investment - 1)*100):.2f}%")

# Save readable rebalance log
readable_rows = []
for log in rebalance_log:
    breakdown_str = json.dumps(log['portfolio_value_breakdown'])
    if log['pl_report']:
        for pl in log['pl_report']:
            readable_rows.append({
                'rebalance_date': log['rebalance_date'],
                'top3': ', '.join([f'{f} ({ret}%)' for f, ret in log['top3']]),
                'bought_funds': ', '.join(log['bought_funds']),
                'sold_funds': ', '.join(log['sold_funds']),
                'sold_fund': pl['fund'],
                'units': round(pl['units'], 2),
                'buy_nav': round(pl['buy_nav'], 2),
                'buy_date': pl['buy_date'],
                'sell_nav': round(pl['sell_nav'], 2) if pl['sell_nav'] is not None else '',
                'sell_date': pl['sell_date'],
                'profit_loss': round(pl['profit_loss'], 2) if pl['profit_loss'] is not None else '',
                'portfolio_value': log['portfolio_value'],
                'portfolio_value_breakdown': breakdown_str
            })
    else:
        readable_rows.append({
            'rebalance_date': log['rebalance_date'],
            'top3': ', '.join([f'{f} ({ret}%)' for f, ret in log['top3']]),
            'bought_funds': ', '.join(log['bought_funds']),
            'sold_funds': ', '.join(log['sold_funds']),
            'sold_fund': '',
            'units': '',
            'buy_nav': '',
            'buy_date': '',
            'sell_nav': '',
            'sell_date': '',
            'profit_loss': '',
            'portfolio_value': log['portfolio_value'],
            'portfolio_value_breakdown': breakdown_str
        })

pd.DataFrame(readable_rows).to_csv('rebalance_log_readable_6m.csv', index=False)
def convert_dates(obj):
    if isinstance(obj, dict):
        return {k: convert_dates(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_dates(i) for i in obj]
    elif hasattr(obj, 'isoformat'):
        return obj.isoformat()
    else:
        return obj

rebalance_log_serializable = convert_dates(rebalance_log)
with open('rebalance_log_readable_6m.json', 'w', encoding='utf-8') as f:
    json.dump(rebalance_log_serializable, f, indent=2, ensure_ascii=False)
print('Readable rebalance log saved to rebalance_log_readable_6m.csv and rebalance_log_readable_6m.json')



Final portfolio value as of 2025-06-13: 3035690.67 INR
Total return: 203.57%
Readable rebalance log saved to rebalance_log_readable_6m.csv and rebalance_log_readable_6m.json
